## Word2Vec

In [1]:
!pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 975.2 kB/s eta 0:00:25
   --- ------------------------------------ 2.2/24.0 MB 20.0 MB/s eta 0:00:02
   ---------- ----------------------------- 6.3/24.0 MB 40.3 MB/s eta 0:00:01
   ------------------- -------------------- 11.9/24.0 MB 108.8 MB/s eta 0:00:01
   -------------------------- ------------- 16.0/24.0 MB 110.0 MB/s eta 0:00:01
   ----------------------------- ---------- 17.8/24.0 MB 108.8 MB/s eta 0:00:01
   ------------------------------ --------- 18.2/24.0 MB 59.5 MB/s eta 0:00:01
   ------------------------------------- -- 22.6/24.0 MB 54.4 MB/s eta 0:00:01
   ---------------------------------------  24.0/24.0 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 43.7 MB/s eta 0:00:00


In [2]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
import gensim
from konlpy.tag import Okt

In [3]:
#  Nsmc 데이터 로드
train_df = pd.read_csv('./data/ratings_train.csv')
train_df.dropna(inplace=True)

기본 전처리 과정

1. 한글만 남김 (특수 기호 제거)
2. 명사만 추출

In [31]:
import re 

train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", " ", regex=True)
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [14]:
okt = Okt()

In [32]:
# 데이터 확인
train_df['document']

0                                       아 더빙   진짜 짜증나네요 목소리
1                         흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼   솔직히 재미는 없다  평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지   소는 뭔죄인가  
149996                                        평점이 너무 낮아서   
149997                      이게 뭐요  한국인은 거들먹거리고 필리핀 혼혈은 착하다 
149998                          청춘 영화의 최고봉 방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [33]:
# 불용어 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '을', '를', '는', '으로']

In [34]:
# 형태소 분석 및 불용어 제거하여 데이터 전처리하기
# 1. 문서 하나씩 읽어오기 (50개만)
# 2. 문서에 okt 형태소 분석 적용 (morphs함수, stem=True 변경)
# 3. 불용어 제거
# 4. tokenized_words 리스트(문서 별 형태소 추출): [[words, words, words], [words, words, words]]
# 5. sentence_nouns 리스트(모든 명사 리스트): [noun, noun, noun, noun]
okt = Okt()

tokenized_words, sentence_nouns = [] , []

for i in train_df['document'][:50]:
    #문서 별 형태소 추출
    tokenized_words = okt.morphs(i, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_words if not word in stopwords]
    tokenized_words.append(stopwords_removed_sentence)
    #명사만 추출
    nouns = okt.nouns(i)
    sentence_nouns.append(nouns)

print(tokenized_words)
print("-------------")
print(sentence_nouns)


['데', '너', '리스', '타르', '가르다', '나다', '용의', '주인', '이', '되다', '싶다', '누', '이랑', '근친상간', '이나', '하다', '다니다', '소설', '속', '에선', '제일', '멋지다', '놈', '이', '자', '이', '메', '라', '니스', '터', '이다', '드라마', '속', '에선', '드래곤', '용', '이', '제일', '멋지다', '웃음', '감독', '님', '토르', '다크', '월드', '는', '말', '아', '잡수다', '기본', '선방', '은', '하다', ['데', '너', '리스', '타르', '가르다', '나다', '용의', '주인', '되다', '싶다', '누', '이랑', '근친상간', '이나', '하다', '다니다', '소설', '속', '에선', '제일', '멋지다', '놈', '자', '메', '라', '니스', '터', '이다', '드라마', '속', '에선', '드래곤', '용', '제일', '멋지다', '웃음', '감독', '님', '토르', '다크', '월드', '말', '아', '잡수다', '기본', '선방', '하다']]
-------------
[['더빙', '진짜', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기'], ['무재', '밓었', '다그', '래서', '추천'], ['교도소', '이야기', '구먼', '재미', '평점', '조정'], ['몬페', '의', '연기', '영화', '스파이더맨', '커스틴', '던스트'], ['막', '걸음', '마', '세', '초등학교', '학년', '생인', '영화', '반개', '움'], ['원작', '긴장감', '제대로'], ['별', '반개', '욕', '이응경', '길용우', '생활', '년', '정말', '발', '해도', '그것', '납치', '감금', '반복', '반복', '드라마', '가족', '연기', '사람', '네'], ['액션',

## 자연어처리에서 데이터 EDA하기

1. 텍스트 데이터의 수를 기준(통계적 수치 확인)  
    1. 텍스트 길이의 최대, 최소, 중앙값, 평균값을 확인  
        - 최대값 : 임베딩을 할 때, max_len 파라미터에 대한 값을 설정
    2. 각 품사별 데이터 수 확인
        - 대부분은 명사가 많이 사용됨
        - 만약 하드웨어등의 사양이 낮아 문제가 되는 경우, 명사만 분석을 하기 위해 데이터 수 확인  
  
2. 용어(텍스트) 기준  
    1. 불용어 사용할 단어
    2. 유의어들 확인하기 위한 원문 데이터 확인

In [35]:
# 리뷰의 최대 길이
word_len = [len(l) for l in tokenized_words]
nouns_len = [len(l) for l in sentence_nouns]

print(f"가장 긴 문서 길이 (단어기준) : {max(word_len)}, 가장 짧은 문서 길이 (단어기준) : {min(word_len)}")

print(f"가장 긴 문서 길이 (명사기준) : {max(nouns_len)}, 가장 짧은 문서 길이 (명사기준) : {min(nouns_len)}")








# '''
# 결과
# 가장 긴 문서 길이 : 95, 가장 짧은 단어 길이 : 1






# 가장 긴 명사 길이 : 66, 가장 짧은 명사 길이 : 0
# '''


가장 긴 문서 길이 (단어기준) : 47, 가장 짧은 문서 길이 (단어기준) : 1
가장 긴 문서 길이 (명사기준) : 29, 가장 짧은 문서 길이 (명사기준) : 1


## Word2Vec 훈련

vector_size = 워드 벡터의 특징 값, 즉 임베딩 된 벡터의 차원  
window = 컨텐스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한(빈도가 적은 단어들을 학습하지 않도록 하는 기준)  
workers = 학습을 위한 프로세스 수 (workers로 설정을 하거나 혹은 multiprocessing으로 병렬처리)  
sg = 0 (CBOW) , 1 (Skip-gram)  

In [ ]:
# word2vec 모델 훈련하기
model_cbow = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5,
         workers=4, sg=0)


model_skipgram = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5,
         workers=4, sg=1)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인(단어수, 차원수)
print(model_cbow.wv.vectors.shape)
print(model_skipgram.wv.vectors.shape)



'''
결과
(15204, 100)
(15204, 100)
'''

모델 저장 방법

1. 재훈련 없이 모델만 사용
2. 재훈련을 할 수 있도록 임베딩에 대한 기본 정보를 함께 저장

In [ ]:
# 모델 저장 방법 1 : 재훈련을 하지 않는 경우
## Ram에 로드하지 않고도 디스크나 네트워크에서 데이터를 즉시 읽어 반복할 수 있음
## 모델 inference 에서 활용하면 좋음

model_cbow.wv.save_word2vec_format('./data/kor_w2v_cbow')
model_skipgram.wv.save_word2vec_format('./data/kor_w2v_skipgram')

In [37]:
# 모델 저장 방법1의 결과를 다시 로드해서 사용

from gensim.models import KeyedVectors

model_cbow_kv = KeyedVectors.load_word2vec_format('./data/kor_w2v_cbow')
model_skipgram_kv = KeyedVectors.load_word2vec_format('./data/kor_w2v_skipgram')




# 모델 불러오기

In [40]:
# 특정 단어를 중심으로 유사한 단어 확인하기
print(model_cbow_kv.most_similar("설경구", topn=5))
print(model_skipgram_kv.most_similar("설경구", topn=5))
#비슷한 단어를 추출하고, 유사도를 측정해준다!


[('김혜수', 0.9176788330078125), ('한석규', 0.9148017168045044), ('황정민', 0.9026427268981934), ('안성기', 0.9024378657341003), ('덴젤', 0.8985658288002014)]
[('한석규', 0.8817006349563599), ('문채원', 0.8791478276252747), ('안성기', 0.875672459602356), ('이주승', 0.862896740436554), ('조재현', 0.8615970015525818)]


In [ ]:
# 모델 저장 방법2 : 재훈련 가능한 모델

model_cbow.save('./data/kor_w2v_cbow.model')
model_skipgram.save('./data/kor_w2v_skipgram.model')



In [41]:
# 모델 불러오기
## 모델에 vocab 정보 등이 함께 저장된 결과를 로드함


model_cbow_model = Word2Vec.load('./data/kor_w2v_cbow.model')
model_skipgram_model = Word2Vec.load('./data/kor_w2v_skipgram.model')

In [45]:
# 특정 단어를 중심으로 유사한 단어 확인하기
print(model_cbow_model.wv.most_similar("추"))
print(model_skipgram_model.wv.most_similar("추"))

[('추강', 0.8743879199028015), ('뻐', 0.8570932745933533), ('Good', 0.8364663124084473), ('파이팅', 0.8234995007514954), ('Very', 0.8139578700065613), ('홧팅', 0.8117495775222778), ('역쉬', 0.8104362487792969), ('good', 0.80389404296875), ('강', 0.8033527731895447), ('짱짱맨', 0.8007481694221497)]
[('추강', 0.8798815608024597), ('강츄', 0.7837347388267517), ('개강', 0.7676759362220764), ('원츄', 0.7452652454376221), ('추하다', 0.7406888008117676), ('강추', 0.7364051938056946), ('요강', 0.734803318977356), ('몬스타', 0.7328289151191711), ('굿굿굿', 0.7328191995620728), ('Good', 0.7307455539703369)]


In [64]:
# 두 단어 간의 유사도 파악하기

model_skipgram_model.wv.most_similar('웅')

[('이기광', 0.909092366695404),
 ('송재림', 0.9026839733123779),
 ('지현', 0.9005348682403564),
 ('소희', 0.8992056846618652),
 ('승호', 0.8982767462730408),
 ('정용화', 0.8953133225440979),
 ('두준', 0.8944463729858398),
 ('박지성', 0.8944243788719177),
 ('찬열', 0.8933141827583313),
 ('**', 0.8929346799850464)]

In [ ]:
# 사전에 없는 경우 확인

In [68]:
# 여러 단어와 유사한 embedding 가져오기

model_skipgram_model.wv.most_similar(positive=['여자', '감독'], negative=['남자'])




[('김기덕', 0.6263550519943237),
 ('역량', 0.6012963652610779),
 ('제작자', 0.5873256921768188),
 ('장진', 0.5859318971633911),
 ('박찬욱', 0.5739709138870239),
 ('자질', 0.5664553046226501),
 ('작가', 0.5588863492012024),
 ('오우삼', 0.5537692308425903),
 ('봉준호', 0.5470575094223022),
 ('임권택', 0.542576253414154)]

In [69]:
# cosmul 사용해 보기
model_skipgram_model.wv.most_similar(positive=['남자', '감독'], negative=['여자'])


[('김기덕', 0.6410436630249023),
 ('장진', 0.6115475296974182),
 ('봉준호', 0.5934121608734131),
 ('천재', 0.5925087928771973),
 ('영화감독', 0.5785976648330688),
 ('핀처', 0.5773937106132507),
 ('린치', 0.5688480138778687),
 ('능력', 0.5647576451301575),
 ('서극', 0.5586300492286682),
 ('여균동', 0.5543100833892822)]

In [74]:
# 그룹간의 유사도 측정
model_cbow_model.wv.most_similar_cosmul(positive=['여자', '감독'])

print(model_skipgram_model.wv.n_similarity(['남자', '배우'], ['여자', '감독']))

print(model_skipgram_model.wv.n_similarity(['남자', '배우'], ['남자', '감독']))

[('놈', 0.515007734298706),
 ('성격', 0.5138322710990906),
 ('교수', 0.5029890537261963),
 ('남자', 0.4960451126098633),
 ('남편', 0.49485912919044495),
 ('본인', 0.49312278628349304),
 ('짓', 0.49199169874191284),
 ('자기', 0.49119845032691956),
 ('직업', 0.49061641097068787),
 ('목사', 0.4866125285625458)]

In [77]:
# 가장 유사하지 않은 단어를 추출
print(model_skipgram_model.wv.doesnt_match(['영화', '드라마', '감독', '선생님']))

print(model_skipgram_model.wv.doesnt_match(['냉장고', '음식', '밥', '당근']))

선생님
음식


기존에 학습된 모델 확인 및 사용

In [78]:
import gensim.downloader as api
from pprint import pprint as pp


list(gensim.downloader.info()['models'].keys())

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

In [79]:
# vector load (로드 시간 있음)

glove_vectors_25 = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [80]:
# 유사도 확인하기

glove_vectors_25.most_similar("twitter")


[('facebook', 0.9480050802230835),
 ('tweet', 0.9403423070907593),
 ('fb', 0.9342359900474548),
 ('instagram', 0.9104824066162109),
 ('chat', 0.8964963555335999),
 ('hashtag', 0.8885936737060547),
 ('tweets', 0.8878158330917358),
 ('tl', 0.8778461217880249),
 ('link', 0.8778210878372192),
 ('internet', 0.8753897547721863)]

In [81]:
glove_vectors_25.most_similar(positive=['woman', 'king'])

[('lady', 0.9231982827186584),
 ('called', 0.9227929711341858),
 ("'s", 0.9222200512886047),
 ('mother', 0.91953045129776),
 ('and', 0.9168232083320618),
 ('young', 0.9150474071502686),
 ('the', 0.9141177535057068),
 ('kid', 0.912209689617157),
 ('guy', 0.9105256199836731),
 ('child', 0.9063714742660522)]

In [84]:
glove_vectors_100 = api.load('glove-twitter-100')


[==================================================] 100.0% 387.1/387.1MB downloaded


In [86]:
glove_vectors_100.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7052315473556519),
 ('prince', 0.6666139960289001),
 ('mother', 0.6436764597892761),
 ('royal', 0.6417251229286194),
 ('father', 0.5952689051628113),
 ('african', 0.5883978009223938),
 ('princess', 0.588217556476593),
 ('called', 0.5842776298522949),
 ('meets', 0.584027886390686),
 ('american', 0.5815179347991943)]

In [88]:
# glove_vectors_100.similar_by_word("cat")
glove_vectors_100.n_similarity(['sushi', 'shop'], ['japanese', 'restaruant'])


0.52202815